In [121]:
import pandas as pd
from sklearn import datasets # sklearn comes with some toy datasets to practise
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import numpy as np
import matplotlib.pyplot as plt
import pickle

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import config
import random

In [122]:
#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

In [123]:
fin_df = pd.read_csv("./fin_df.csv", index_col=0)

In [124]:
hot_songs = pd.read_csv("./df.csv", index_col=0)

In [125]:
fin_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,names,artists
0,0.533,0.638,4,-7.130,1,0.0341,0.2020,0.000000,0.0729,0.937,156.663,audio_features,3ohLnESFgYACPMCkoTOzqE,spotify:track:3ohLnESFgYACPMCkoTOzqE,https://api.spotify.com/v1/tracks/3ohLnESFgYAC...,https://api.spotify.com/v1/audio-analysis/3ohL...,153760,4,The Twist,['Chubby Checker']
1,0.612,0.924,9,-3.877,1,0.0316,0.1590,0.000043,0.1030,0.960,115.953,audio_features,3i3GeK0qLQybu4ah42YmCY,spotify:track:3i3GeK0qLQybu4ah42YmCY,https://api.spotify.com/v1/tracks/3i3GeK0qLQyb...,https://api.spotify.com/v1/audio-analysis/3i3G...,295987,4,Smooth (feat. Rob Thomas),"['Santana', 'Rob Thomas']"
2,0.741,0.727,5,-4.174,0,0.1510,0.0205,0.000000,0.2640,0.370,130.023,audio_features,70dWrqAp30TmWeibQkn0i7,spotify:track:70dWrqAp30TmWeibQkn0i7,https://api.spotify.com/v1/tracks/70dWrqAp30Tm...,https://api.spotify.com/v1/audio-analysis/70dW...,263360,4,Party Rock Anthem,"['LMFAO', 'Lauren Bennett', 'GoonRock']"
3,0.597,0.463,7,-5.528,1,0.0371,0.8240,0.000000,0.1070,0.227,115.837,audio_features,41DYHkqBtk7LawsUUnzAMW,spotify:track:41DYHkqBtk7LawsUUnzAMW,https://api.spotify.com/v1/tracks/41DYHkqBtk7L...,https://api.spotify.com/v1/audio-analysis/41DY...,229840,4,You Were Meant For Me,['Jewel']
4,0.743,0.766,0,-6.375,1,0.0265,0.0873,0.000000,0.5090,0.610,127.960,audio_features,4vp2J1l5RD4gMZwGFLfRAu,spotify:track:4vp2J1l5RD4gMZwGFLfRAu,https://api.spotify.com/v1/tracks/4vp2J1l5RD4g...,https://api.spotify.com/v1/audio-analysis/4vp2...,289133,4,I Gotta Feeling,['Black Eyed Peas']


### scaling features

In [126]:
fin_df.describe()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
count,4937.000000,4937.000000,4937.000000,4937.000000,4937.000000,4937.000000,4937.000000,4937.000000,4937.000000,4937.000000,4937.000000,4.937000e+03,4937.000000
mean,0.605546,0.598410,5.208224,-9.171596,0.733036,0.061127,0.329579,0.032453,0.181293,0.639191,119.303663,2.223860e+05,3.945716
std,0.152746,0.203118,3.555930,3.525859,0.442418,0.063598,0.286095,0.136872,0.149510,0.238533,27.584677,6.587229e+04,0.275061
min,0.156000,0.026400,0.000000,-23.954000,0.000000,0.022000,0.000025,0.000000,0.014600,0.038700,40.120000,8.910700e+04,1.000000
25%,0.507000,0.452000,2.000000,-11.548000,0.000000,0.031400,0.069100,0.000000,0.086600,0.458000,99.201000,1.718270e+05,4.000000
50%,0.618000,0.609000,5.000000,-8.794000,1.000000,0.038700,0.246000,0.000008,0.125000,0.677000,117.907000,2.195730e+05,4.000000
75%,0.715000,0.757000,8.000000,-6.482000,1.000000,0.058000,0.572000,0.000667,0.232000,0.845000,133.570000,2.588270e+05,4.000000
max,0.988000,0.996000,11.000000,-1.040000,1.000000,0.898000,0.985000,0.977000,0.992000,0.993000,231.709000,1.055400e+06,5.000000


In [127]:
#get numercial features

X = fin_df.select_dtypes(include=np.number)

In [128]:
X.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0.533,0.638,4,-7.130,1,0.0341,0.2020,0.000000,0.0729,0.937,156.663,153760,4
1,0.612,0.924,9,-3.877,1,0.0316,0.1590,0.000043,0.1030,0.960,115.953,295987,4
2,0.741,0.727,5,-4.174,0,0.1510,0.0205,0.000000,0.2640,0.370,130.023,263360,4
3,0.597,0.463,7,-5.528,1,0.0371,0.8240,0.000000,0.1070,0.227,115.837,229840,4
4,0.743,0.766,0,-6.375,1,0.0265,0.0873,0.000000,0.5090,0.610,127.960,289133,4


In [131]:
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns = X.columns)
display(X.head())
print()
display(X_scaled_df.head())

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0.533,0.638,4,-7.130,1,0.0341,0.2020,0.000000,0.0729,0.937,156.663,153760,4
1,0.612,0.924,9,-3.877,1,0.0316,0.1590,0.000043,0.1030,0.960,115.953,295987,4
2,0.741,0.727,5,-4.174,0,0.1510,0.0205,0.000000,0.2640,0.370,130.023,263360,4
3,0.597,0.463,7,-5.528,1,0.0371,0.8240,0.000000,0.1070,0.227,115.837,229840,4
4,0.743,0.766,0,-6.375,1,0.0265,0.0873,0.000000,0.5090,0.610,127.960,289133,4


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,-0.474995,0.194930,-0.339811,0.579094,0.603481,-0.425015,-0.445977,-0.237127,-0.725066,1.248629,1.354488,-1.041909,0.197372
1,0.042256,1.603123,1.066433,1.501799,0.603481,-0.464329,-0.596292,-0.236810,-0.523721,1.345061,-0.121481,1.117442,0.197372
2,0.886880,0.633144,-0.058563,1.417556,-1.657054,1.413281,-1.080446,-0.237127,0.553241,-1.128645,0.388637,0.622085,0.197372
3,-0.055957,-0.666727,0.503935,1.033497,0.603481,-0.377839,1.728347,-0.237127,-0.496964,-1.728204,-0.125686,0.113170,0.197372
4,0.899975,0.825170,-1.464807,0.793248,0.603481,-0.544528,-0.846933,-0.237127,2.192097,-0.122391,0.313841,1.013382,0.197372


In [132]:
X_scaled_df.describe() # to check if features have the same weight

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
count,4.937000e+03,4.937000e+03,4.937000e+03,4.937000e+03,4.937000e+03,4.937000e+03,4.937000e+03,4.937000e+03,4.937000e+03,4.937000e+03,4.937000e+03,4.937000e+03,4.937000e+03
mean,1.885265e-16,5.416891e-17,-1.375354e-16,1.087960e-16,1.073208e-15,-6.622659e-17,1.237954e-16,4.230125e-16,2.544495e-17,-1.214342e-18,1.170715e-16,-4.604449e-17,4.859728e-16
std,1.000101e+00,1.000101e+00,1.000101e+00,1.000101e+00,1.000101e+00,1.000101e+00,1.000101e+00,1.000101e+00,1.000101e+00,1.000101e+00,1.000101e+00,1.000101e+00,1.000101e+00
min,-2.943393e+00,-2.816437e+00,-1.464807e+00,-4.192992e+00,-1.657054e+00,-6.152921e-01,-1.152019e+00,-2.371269e-01,-1.115047e+00,-2.517694e+00,-2.870858e+00,-2.023498e+00,-1.071039e+01
25%,-6.452293e-01,-7.208881e-01,-9.023092e-01,-6.740611e-01,-1.657054e+00,-4.674736e-01,-9.105551e-01,-2.371269e-01,-6.334237e-01,-7.596853e-01,-7.288358e-01,-7.676081e-01,1.973722e-01
50%,8.154039e-02,5.214112e-02,-5.856265e-02,1.071041e-01,6.034807e-01,-3.526784e-01,-2.921662e-01,-2.370710e-01,-3.765582e-01,1.585209e-01,-5.063697e-02,-4.270790e-02,1.973722e-01
75%,7.166454e-01,7.808565e-01,7.851839e-01,7.628972e-01,6.034807e-01,-4.917866e-02,8.474313e-01,-2.322533e-01,3.391868e-01,8.628983e-01,5.172358e-01,5.532631e-01,1.973722e-01
max,2.504106e+00,1.957633e+00,1.628930e+00,2.306507e+00,6.034807e-01,1.316014e+01,2.291155e+00,6.901661e+00,5.422983e+00,1.483421e+00,4.075332e+00,1.264717e+01,3.833291e+00


### Clustering

In [133]:
kmeans = KMeans(n_clusters=12, random_state=1234)
kmeans.fit(X_scaled_df)

KMeans(n_clusters=12, random_state=1234)

In [134]:
clusters = kmeans.predict(X_scaled_df)
#clusters
pd.Series(clusters).value_counts().sort_index()

0     608
1     586
2     462
3     562
4     230
5     643
6     152
7     240
8     349
9     484
10    356
11    265
dtype: int64

In [135]:
# check which group was assigend 

X["cluster"] = clusters
X.head()

<ipython-input-135-99329742fc70>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["cluster"] = clusters


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,cluster
0,0.533,0.638,4,-7.130,1,0.0341,0.2020,0.000000,0.0729,0.937,156.663,153760,4,10
1,0.612,0.924,9,-3.877,1,0.0316,0.1590,0.000043,0.1030,0.960,115.953,295987,4,3
2,0.741,0.727,5,-4.174,0,0.1510,0.0205,0.000000,0.2640,0.370,130.023,263360,4,5
3,0.597,0.463,7,-5.528,1,0.0371,0.8240,0.000000,0.1070,0.227,115.837,229840,4,9
4,0.743,0.766,0,-6.375,1,0.0265,0.0873,0.000000,0.5090,0.610,127.960,289133,4,7


### playing with the parameters

In [136]:
kmeans.inertia_ # decided k=12

32128.361059570205

In [137]:
# K = range(2, 21)
# inertia = []

# for k in K:
#     print("Training a K-Means model with {} neighbours! ".format(k))
#     print()
#     kmeans = KMeans(n_clusters=k,
#                     random_state=1234)
#     kmeans.fit(X_scaled_df)
#     inertia.append(kmeans.inertia_)

# import numpy as np
# import matplotlib.pyplot as plt
# %matplotlib inline

# plt.figure(figsize=(16,8))
# plt.plot(K, inertia, 'bx-')
# plt.xlabel('k')
# plt.ylabel('inertia')
# plt.xticks(np.arange(min(K), max(K)+1, 1.0))
# plt.title('Elbow Method showing the optimal k')

elbow-> 8-9 checkin silouhette to decide..

In [138]:
# K = range(2, 20)
# silhouette = []

# for k in K:
#     kmeans = KMeans(n_clusters=k,random_state=1234)
#     kmeans.fit(X_scaled_df)
#     silhouette.append(silhouette_score(X_scaled_df, kmeans.predict(X_scaled_df)))


# plt.figure(figsize=(16,8))
# plt.plot(K, silhouette, 'bx-')
# plt.xlabel('k')
# plt.ylabel('silhouette score')
# plt.xticks(np.arange(min(K), max(K)+1, 1.0))
# plt.title('Silhouette Method showing the optimal k')



In [139]:
#here is the optimal k=3 the results are dissapointed
#for us is the optimal k between 8-9 -> k=12

In [140]:
X.head(3)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,cluster
0,0.533,0.638,4,-7.130,1,0.0341,0.2020,0.000000,0.0729,0.937,156.663,153760,4,10
1,0.612,0.924,9,-3.877,1,0.0316,0.1590,0.000043,0.1030,0.960,115.953,295987,4,3
2,0.741,0.727,5,-4.174,0,0.1510,0.0205,0.000000,0.2640,0.370,130.023,263360,4,5


In [141]:
X['id'] = fin_df['id']

<ipython-input-141-ee962f14bad8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['id'] = fin_df['id']


In [142]:
#save X df  feat_clust.csv
X.to_csv("./feat_clust_id.csv") # X is the scaled features dataframe

In [143]:
feat_clust = X.copy() # rename the DF to avoid confusion

In [144]:
feat_clust.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,cluster,id
0,0.533,0.638,4,-7.130,1,0.0341,0.2020,0.000000,0.0729,0.937,156.663,153760,4,10,3ohLnESFgYACPMCkoTOzqE
1,0.612,0.924,9,-3.877,1,0.0316,0.1590,0.000043,0.1030,0.960,115.953,295987,4,3,3i3GeK0qLQybu4ah42YmCY
2,0.741,0.727,5,-4.174,0,0.1510,0.0205,0.000000,0.2640,0.370,130.023,263360,4,5,70dWrqAp30TmWeibQkn0i7
3,0.597,0.463,7,-5.528,1,0.0371,0.8240,0.000000,0.1070,0.227,115.837,229840,4,9,41DYHkqBtk7LawsUUnzAMW
4,0.743,0.766,0,-6.375,1,0.0265,0.0873,0.000000,0.5090,0.610,127.960,289133,4,7,4vp2J1l5RD4gMZwGFLfRAu


In [145]:
hot_songs.head()

,Rank,Song Title,Song Artist,Year,Genre,Decade
0,57,1999,Prince,1983,Popular,1980
0,20,(Everything I Do) I Do It For You,Bryan Adams,1991,Ballad,1990
0,46,(I Can't Get No) Satisfaction,Rolling Stones,1965,Oldies,1960
0,15,(I've Had) The Time Of My Life,Bill Medley & Jennifer Warnes,1987,Ballad,1980
0,47,(Let Me Be Your) Teddy Bear,Elvis Presley,1957,Oldies,1950


In [146]:
#rename columns:
hot_songs = hot_songs.rename(columns={'Rank': 'rank', 'Song Title': 'title', "Song Artist": "artist", "Year": "year", "Genre": "genre", "Decade": "decade"})

In [147]:
def get_artists_from_id(id_):
    artists = sp.track(id_)['artists']
    artists_lst = [a['name'] for a in artists]
    return ", ".join(artists_lst)

In [148]:
def get_trackname_from_id(id_):
    return sp.track(id_)['name']

In [156]:
# X -> scaled DF (+ cluster labels)

def song_recommender():
    ask_song = input("Tell us a song you like: ").title()
    if hot_songs['title'].str.contains(ask_song).all():
        print("Nice, that is a hot song! Do you know " + hot_songs.sample(n=1)["title"].item() + " by " + hot_songs.sample(n=1)["artist"].item() + "? I'm sure you will like it!")
       

    else:
        print("Hey, that's not a hot song right now. Let me recommend another song for you!")
        results = sp.search(q = ask_song, limit=1)
        if results["tracks"]["items"]: 
            user_uri = results["tracks"]["items"][0]["uri"] #search after uri
            user_audio = sp.audio_features(tracks = user_uri) #get audio features
            user_df = pd.DataFrame(user_audio[0], index=[0])[feat_clust.drop(['cluster','id'], axis=1).columns]
            user_scaled = scaler.transform(user_df) # scale user input
            user_cluster = kmeans.predict(user_scaled)
            print(user_cluster)
            recommend_id = feat_clust[feat_clust['cluster'] == user_cluster[0]].sample(n=1)["id"].item()
            recommend_song_title = get_trackname_from_id(recommend_id)
            print("Nice song! Do you know: " + recommend_song_title.upper() + ", I'm sure you will like it!")
           
        else:
            print("No results.")

In [158]:
song_recommender()

Tell us a song you like: nothing else matters
Hey, that's not a hot song right now. Let me recommend another song for you!
[11]
Nice song! Do you know: IF I CAN DREAM - STEREO MIX, I'm sure you will like it!


In [ ]:
song_recommender()